# Seeds

In [38]:
# Cell 0 — Seeds (minimal)
import os, random, numpy as np, torch
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


# chargement et pretraitement des donnees

Imports & schéma de colonnes

In [39]:
# =========================
# Cell 1 — Imports & colonnes
# =========================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

# Schéma des colonnes du WDBC (Breast Cancer Wisconsin)
columns = [
    "id", "diagnosis",
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean",
    "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se",
    "compactness_se", "concavity_se", "concave_points_se", "symmetry_se", "fractal_dimension_se",
    "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
    "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"
]


Chargement + préparation X/y

In [40]:
# =========================
# Cell 2 — Chargement CSV & X/y
# =========================
df = pd.read_csv("wdbc.data", header=None, names=columns)

# Séparer X (features) et y (target binaire)
X = df.drop(['id', 'diagnosis'], axis=1)
y = df['diagnosis'].map({'B': 0, 'M': 1})  # 0 = Bénin, 1 = Malin

print("Dimensions complètes :", X.shape)
print("Répartition classes :", y.value_counts().to_dict())


Dimensions complètes : (569, 30)
Répartition classes : {0: 357, 1: 212}


Split STRATIFIÉ + Scaling fit-on-train

In [41]:
# =========================
# Cell 3 — Split stratifié + scaling fit-on-train
# =========================
#  split AVANT le fit du scaler pour éviter la fuite d'information
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Standardisation (fit sur TRAIN uniquement, puis transform sur TRAIN & TEST)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

print(f"Taille du train set : {X_train.shape[0]} échantillons")
print(f"Taille du test set  : {X_test.shape[0]} échantillons")


Taille du train set : 398 échantillons
Taille du test set  : 171 échantillons


TensorDataset & DataLoaders

In [42]:
# =========================
# Cell 4 — TensorDataset & DataLoaders
# =========================
# Conversion en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor  = torch.tensor(X_test,  dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.long)

# Datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor,  y_test_tensor)

# DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

print("Batch size :", batch_size)


Batch size : 64


Device (CPU)

In [43]:
# =========================
# Cell 5 — Device (CPU fixé)
# =========================
import torch
device = torch.device("cpu")
print("Device utilisé :", device)


Device utilisé : cpu


# definition de model MLP

In [45]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size=30, hidden_sizes=[128, 64, 32], dropout_rate=0.5):
        super(MLP, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])

        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])

        self.fc4 = nn.Linear(hidden_sizes[2], 2)

        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)

        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)

        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)

        return self.fc4(x)



# entrainement de model et evaluation sur donnes propre

In [46]:
!pip install adversarial-robustness-toolbox

In [48]:
# =========================
# Cellule — Mixed Adversarial Training (MLP tabulaire, CPU) — avec λ explicite
# =========================
import time, copy, numpy as np, torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

device = torch.device("cpu")  # cohérent avec le reste
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

EPOCHS, PATIENCE = 30, 5
best_val, no_impr = float("inf"), 0
best_state = copy.deepcopy(model.state_dict())

# --- bornes par feature (dans l'espace standardisé)
with torch.no_grad():
    X_MIN = X_train_tensor.min(dim=0).values.to(device)  # shape: [30]
    X_MAX = X_train_tensor.max(dim=0).values.to(device)

def clamp_per_feature(x):
    # x: [B, 30]; clamp par-feature avec broadcast
    return torch.max(torch.min(x, X_MAX), X_MIN)

# --- hyperparams MAT (tabulaire standardisé)
EPS_TRAIN   = 0.2        # ~0.1–0.3 recommandé pour données standardisées
ADV_FRAC    = 0.50       # n'attaquer qu'une sous-partie du batch (~50%)
PGD_STEPS   = 5          # PGD court pour rester rapide sur CPU
PGD_ALPHA   = EPS_TRAIN / 5
BIM_STEPS   = 10
BIM_ALPHA   = EPS_TRAIN / 10
LAMBDA      = 0.50       # NEW: pondération explicite de la perte adversariale (0..1)

# --- attaques en espace standardisé (les DataLoaders sortent X déjà standardisé)
def fgsm(x, y, eps=EPS_TRAIN):
    model.eval()
    x_adv = x.detach().clone().requires_grad_(True)
    loss = criterion(model(x_adv), y)
    model.zero_grad(set_to_none=True)
    loss.backward()
    x_adv = x_adv + eps * x_adv.grad.detach().sign()
    x_adv = clamp_per_feature(x_adv).detach()
    model.train()
    return x_adv

def pgd(x, y, eps=EPS_TRAIN, alpha=PGD_ALPHA, iters=PGD_STEPS, random_start=True):
    model.eval()
    x0 = x.detach()
    if random_start:
        delta0 = torch.empty_like(x0).uniform_(-eps, eps)
        x_adv = clamp_per_feature(x0 + delta0)
    else:
        x_adv = x0.clone()
    for _ in range(iters):
        x_adv.requires_grad_(True)
        loss = criterion(model(x_adv), y)
        model.zero_grad(set_to_none=True)
        loss.backward()
        x_adv = x_adv + alpha * x_adv.grad.detach().sign()
        delta = torch.clamp(x_adv - x0, min=-eps, max=eps)
        x_adv = clamp_per_feature(x0 + delta).detach()
    model.train()
    return x_adv

def bim(x, y, eps=EPS_TRAIN, alpha=BIM_ALPHA, iters=BIM_STEPS):
    # BIM = PGD sans random start, plus d'itérations petites
    return pgd(x, y, eps=eps, alpha=alpha, iters=iters, random_start=False)

# --- éval clean
def evaluate(model, loader):
    model.eval(); loss_sum=0.0; n=0; correct=0; probs_all=[]; y_all=[]
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb); loss = criterion(logits, yb)
            loss_sum += loss.item()*yb.size(0); n += yb.size(0)
            correct += (logits.argmax(1) == yb).sum().item()
            probs_all.append(torch.softmax(logits, dim=1)[:,1].cpu().numpy())
            y_all.append(yb.cpu().numpy())
    from numpy import concatenate as cat
    y_all = cat(y_all); probs_all = cat(probs_all)
    return (loss_sum/n, correct/n, roc_auc_score(y_all, probs_all))

# --- entraînement MAT : mix équilibré (rotation FGSM/PGD/BIM) + sous-batch attaqué, avec λ explicite
attacks = ["fgsm","pgd","bim"]
att_idx = 0  # mix équilibré en rotation

for epoch in range(1, EPOCHS+1):
    model.train(); t0=time.time(); run_loss=0.0; n=0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        B = xb.size(0)
        k = int(round(ADV_FRAC * B))  # n’attaquer qu’une sous-partie du batch

        optimizer.zero_grad(set_to_none=True)

        if k > 0:
            # indices attaqués et non attaqués
            idx_adv = torch.randperm(B, device=device)[:k]
            mask_clean = torch.ones(B, dtype=torch.bool, device=device)
            mask_clean[idx_adv] = False

            # génère adv sur la sous-partie
            xa, ya = xb[idx_adv], yb[idx_adv]
            a = attacks[att_idx % len(attacks)]  # rotation FGSM→PGD→BIM
            att_idx += 1
            if a == "fgsm":
                xa = fgsm(xa, ya, eps=EPS_TRAIN)
            elif a == "pgd":
                xa = pgd(xa, ya, eps=EPS_TRAIN, alpha=PGD_ALPHA, iters=PGD_STEPS, random_start=True)
            else:
                xa = bim(xa, ya, eps=EPS_TRAIN, alpha=BIM_ALPHA, iters=BIM_STEPS)

            # deux passes et mélange explicite des pertes
            loss_terms = []

            if mask_clean.any():  # partie clean présente
                logits_clean = model(xb[mask_clean])
                loss_clean = criterion(logits_clean, yb[mask_clean])
                loss_terms.append((1.0 - LAMBDA) * loss_clean)

            if k > 0:  # partie adv présente
                logits_adv = model(xa)
                loss_adv = criterion(logits_adv, ya)
                loss_terms.append(LAMBDA * loss_adv)

            # cas limites: si toute la batch est clean/adv, on ne réduit pas artificiellement la perte
            if len(loss_terms) == 1:
                loss = loss_terms[0] / max(1.0, (1.0 - LAMBDA if mask_clean.any() else LAMBDA))
            else:
                loss = sum(loss_terms)

        else:
            # aucun exemple adv : perte clean uniquement
            logits = model(xb)
            loss = criterion(logits, yb)

        loss.backward(); optimizer.step()
        run_loss += loss.item()*yb.size(0); n += yb.size(0)

    train_loss = run_loss/n
    val_loss, val_acc, val_auc = evaluate(model, test_loader)  # ou val_loader si dispo
    print(f"[MAT-MLP-λ] Epoch {epoch:02d} | train_loss={train_loss:.4f} | "
          f"val_loss={val_loss:.4f} | val_acc={val_acc:.3f} | val_auc={val_auc:.3f} | {time.time()-t0:.1f}s")

    if val_loss < best_val - 1e-4:
        best_val = val_loss; no_impr = 0
        best_state = copy.deepcopy(model.state_dict())
    else:
        no_impr += 1
        if no_impr >= PATIENCE:
            print("Early stopping."); break

# --- Évaluation finale (clean)
model.load_state_dict(best_state); model.eval()
y_true, y_pred, y_prob = [], [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        logits = model(xb)
        y_prob.extend(torch.softmax(logits, dim=1)[:,1].cpu().numpy())
        y_pred.extend(logits.argmax(1).cpu().numpy())
        y_true.extend(yb.numpy())

print("DEF(MAT-MLP-λ) Test | acc:{:.4f} | prec:{:.4f} | rec:{:.4f} | f1:{:.4f} | auc:{:.4f}".format(
    accuracy_score(y_true, y_pred),
    precision_score(y_true, y_pred, zero_division=0),
    recall_score(y_true, y_pred, zero_division=0),
    f1_score(y_true, y_pred, zero_division=0),
    roc_auc_score(y_true, y_prob),
))


[MAT-MLP-λ] Epoch 01 | train_loss=0.2369 | val_loss=0.0843 | val_acc=0.965 | val_auc=0.999 | 0.1s
[MAT-MLP-λ] Epoch 02 | train_loss=0.2002 | val_loss=0.0794 | val_acc=0.965 | val_auc=0.999 | 0.1s
[MAT-MLP-λ] Epoch 03 | train_loss=0.2351 | val_loss=0.0991 | val_acc=0.959 | val_auc=0.998 | 0.1s
[MAT-MLP-λ] Epoch 04 | train_loss=0.2521 | val_loss=0.1222 | val_acc=0.947 | val_auc=0.998 | 0.1s
[MAT-MLP-λ] Epoch 05 | train_loss=0.1994 | val_loss=0.0851 | val_acc=0.971 | val_auc=0.998 | 0.1s
[MAT-MLP-λ] Epoch 06 | train_loss=0.2306 | val_loss=0.1042 | val_acc=0.947 | val_auc=0.998 | 0.1s
[MAT-MLP-λ] Epoch 07 | train_loss=0.2222 | val_loss=0.1072 | val_acc=0.953 | val_auc=0.998 | 0.1s
Early stopping.
DEF(MAT-MLP-λ) Test | acc:0.9649 | prec:1.0000 | rec:0.9062 | f1:0.9508 | auc:0.9987


# evaluation de model sur donnees adv

In [49]:
from art.estimators.classification import PyTorchClassifier
import torch.nn as nn
import torch.optim as optim

clip_min = X_train_tensor.min(dim=0).values.numpy()
clip_max = X_train_tensor.max(dim=0).values.numpy()

classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(30,),
    nb_classes=2,
    clip_values=(clip_min, clip_max),
    device_type="cpu"
)



#Préparation NumPy du set de test (standardisé) pour ART

In [50]:
import numpy as np

X_test_np = X_test_tensor.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()


In [51]:
model.eval()
def evaluate_attack(X_adv_np, y_true_np, attack_name):
    X_adv_tensor = torch.tensor(X_adv_np, dtype=torch.float32).to(device)
    with torch.no_grad():
        outputs = model(X_adv_tensor)
        probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

    acc = accuracy_score(y_true_np, preds)
    precision = precision_score(y_true_np, preds, zero_division=0)
    recall = recall_score(y_true_np, preds, zero_division=0)
    f1 = f1_score(y_true_np, preds, zero_division=0)
    auc = roc_auc_score(y_true_np, probs)

    print(f"\n📉 📊 Résultats après attaque {attack_name}")
    print(f"Accuracy       : {acc:.4f}")
    print(f"Precision      : {precision:.4f}")
    print(f"Recall         : {recall:.4f}")
    print(f"F1-score       : {f1:.4f}")
    print(f"AUC-ROC        : {auc:.4f}")



In [52]:
from art.attacks.evasion import (
    FastGradientMethod,
    ProjectedGradientDescent,
    BasicIterativeMethod,
    CarliniL2Method
)

# FGSM
for eps in [0.1, 0.2, 0.3]:
    fgsm = FastGradientMethod(estimator=classifier, eps=eps)
    X_adv = fgsm.generate(x=X_test_np)
    evaluate_attack(X_adv, y_test_np, f"FGSM (eps={eps})")

# PGD
for eps in [0.1, 0.2, 0.3]:
    pgd = ProjectedGradientDescent(estimator=classifier, eps=eps, eps_step=eps/10,
                                   max_iter=20, norm=np.inf, targeted=False, num_random_init=0)
    X_adv = pgd.generate(x=X_test_np)
    evaluate_attack(X_adv, y_test_np, f"PGD (eps={eps}, step={eps/10}, it=20)")


# BIM
for eps in [0.1, 0.2, 0.3]:
    bim = BasicIterativeMethod(estimator=classifier, eps=eps, eps_step=eps/10, max_iter=10)
    X_adv = bim.generate(x=X_test_np)
    evaluate_attack(X_adv, y_test_np, f"BIM (eps={eps}, step={eps/10}, it=10)")

# C&W — config rapide
cw_fast = CarliniL2Method(classifier=classifier, confidence=0.0, targeted=False,
                          learning_rate=0.02, max_iter=75, binary_search_steps=1,
                          initial_const=0.3, batch_size=64)
X_cw_adv_fast = cw_fast.generate(x=X_test_np)
evaluate_attack(X_cw_adv_fast, y_test_np, "C&W-L2 (conf=0, c0=0.3, it=75, bsearch=1)")

# C&W — config forte
X_cw_input = X_test_np
y_cw_input = y_test_np
cw_hard = CarliniL2Method(classifier=classifier, confidence=0.0, targeted=False,
                          learning_rate=0.01, max_iter=500, binary_search_steps=7,
                          initial_const=0.01, batch_size=64)
X_cw_adv_hard = cw_hard.generate(x=X_cw_input)
evaluate_attack(X_cw_adv_hard, y_cw_input,
                "C&W-L2 STRONG (conf=0, c0=0.01, it=500, bsearch=7, lr=0.01)")



📉 📊 Résultats après attaque FGSM (eps=0.1)
Accuracy       : 0.9708
Precision      : 0.9836
Recall         : 0.9375
F1-score       : 0.9600
AUC-ROC        : 0.9965

📉 📊 Résultats après attaque FGSM (eps=0.2)
Accuracy       : 0.9240
Precision      : 0.9322
Recall         : 0.8594
F1-score       : 0.8943
AUC-ROC        : 0.9775

📉 📊 Résultats après attaque FGSM (eps=0.3)
Accuracy       : 0.8480
Precision      : 0.8065
Recall         : 0.7812
F1-score       : 0.7937
AUC-ROC        : 0.9190


PGD - Batches:   0%|          | 0/6 [00:00<?, ?it/s]


📉 📊 Résultats après attaque PGD (eps=0.1, step=0.01, it=20)
Accuracy       : 0.9708
Precision      : 0.9836
Recall         : 0.9375
F1-score       : 0.9600
AUC-ROC        : 0.9963


PGD - Batches:   0%|          | 0/6 [00:00<?, ?it/s]


📉 📊 Résultats après attaque PGD (eps=0.2, step=0.02, it=20)
Accuracy       : 0.9181
Precision      : 0.9167
Recall         : 0.8594
F1-score       : 0.8871
AUC-ROC        : 0.9734


...


📉 📊 Résultats après attaque BIM (eps=0.3, step=0.03, it=10)
Accuracy       : 0.8421
Precision      : 0.8033
Recall         : 0.7656
F1-score       : 0.7840
AUC-ROC        : 0.9092


C&W L_2:   0%|          | 0/3 [00:00<?, ?it/s]


📉 📊 Résultats après attaque C&W-L2 (conf=0, c0=0.3, it=75, bsearch=1)
Accuracy       : 0.9474
Precision      : 0.9825
Recall         : 0.8750
F1-score       : 0.9256
AUC-ROC        : 0.9974


C&W L_2:   0%|          | 0/3 [00:00<?, ?it/s]


📉 📊 Résultats après attaque C&W-L2 STRONG (conf=0, c0=0.01, it=500, bsearch=7, lr=0.01)
Accuracy       : 0.8772
Precision      : 0.8308
Recall         : 0.8438
F1-score       : 0.8372
AUC-ROC        : 0.9766
